In [1]:
from read_edf import plot_signals
from read_result import read_result_file
from read_result import plot_sleep_stages
from split_signal import split_edf_by_annotations
from split_signal import split_edf_by_30s
from sklearn.decomposition import PCA
import processing_data
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
import svm_builder
import importlib

In [2]:
feature_number = 105

X = np.empty((0,feature_number))
y = np.empty((0,))
for dirname, _, filenames in os.walk('./result'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        file = np.load(file_path)
        X = np.concatenate((X, file['features']))
        y = np.concatenate((y, file['labels']))
print(X.shape)
print(y.shape)

(393345, 105)
(393345,)


In [3]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.fit_transform(X)

In [4]:
pca1 = PCA(n_components= 0.999).fit(X_scaled)
pca2 = PCA(n_components= 0.95).fit(X_scaled)
X_pca1 = pca1.transform(X_scaled)
X_pca2 = pca2.transform(X_scaled)
print(X_pca1.shape)
print(X_pca2.shape)

(393345, 82)
(393345, 46)


In [5]:
model_X = X_pca1
model_y = y
print(model_X.shape, model_y.shape)

(393345, 82) (393345,)


In [9]:
from sklearn.model_selection import train_test_split

In [11]:
# devide X and y into train dataset and test dataset:
X_train, X_test, y_train, y_test = train_test_split(model_X, model_y, test_size=0.3, random_state=42)

In [12]:
import logistic_builder

In [13]:
log_model = logistic_builder.train_logistic_model(X_train, y_train)

Training set score of logistic: 0.8915159439238759
Testing set score of logistic: 0.8916746365144123


In [47]:
y_pred = log_model.predict(X_test)
y_pred = y_pred.astype(int).tolist()
y_test = y_test.astype(int).tolist()


In [55]:
import scoring
importlib.reload(scoring)

<module 'scoring' from 'E:\\2023 Fall\\IML\\project\\iml-project\\scoring.py'>

In [56]:
scoring.score(y_pred, y_test)

Class 0: Precision: 0.952, Recall: 0.985
Class 1: Precision: 0.426, Recall: 0.116
Class 2: Precision: 0.720, Recall: 0.892
Class 3: Precision: 0.424, Recall: 0.038
Class 4: Precision: 0.855, Recall: 0.634
Class 5: Precision: 0.756, Recall: 0.545
Class 6: Precision: 0.724, Recall: 0.070
mf1 score: 0.559
acc_score: 0.890
kappa_score: 0.953


In [50]:
cnt = len(y_test)
acc = np.zeros(7)
for i in range(cnt):
    if y_test[i] == y_pred[i]:
        acc[int(y_test[i])] += 1
for i in range(7):
    print(f"Accuracy of class {i}: {acc[i]/y_test.count(i)}")

Accuracy of class 0: 0.9845500788503266
Accuracy of class 1: 0.11566927468809474
Accuracy of class 2: 0.8919277044575185
Accuracy of class 3: 0.03796653796653797
Accuracy of class 4: 0.634313725490196
Accuracy of class 5: 0.544734648513476
Accuracy of class 6: 0.07023411371237458
